In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime
import numpy as np
import yfinance as yf

In [2]:
conn = mysql.connector.connect(
    host='localhost',
    port=3306,
    database='options',
    user='root',
    password=''  
    )

spy_query = """
SELECT 
    date,
    expiration,
    strike,
    call_put,
    bid,
    ask,
    vol,
    delta,
    gamma,
    theta,
    vega,
    rho
FROM option_chain 
WHERE act_symbol = 'SPY'
ORDER BY date DESC, expiration, strike
"""

spy_df = pd.read_sql(spy_query, conn)

/var/folders/lg/z6wm11r5617_11hqzqgtr_vw0000gn/T/ipykernel_13882/3222874332.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spy_df = pd.read_sql(spy_query, conn)


In [ ]:
spy_df

,date,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho
0,2025-07-21,2025-08-01,440.0,Call,188.55,190.28,0.6782,0.9988,0.0000,-0.0533,0.0035,0.1322
1,2025-07-21,2025-08-01,440.0,Put,0.01,0.02,0.6782,-0.0009,0.0000,-0.0107,0.0035,-0.0001
2,2025-07-21,2025-08-01,470.0,Call,158.61,160.61,0.5727,0.9984,0.0000,-0.0589,0.0049,0.1412
3,2025-07-21,2025-08-01,470.0,Put,0.02,0.03,0.5727,-0.0013,0.0000,-0.0126,0.0049,-0.0002
4,2025-07-21,2025-08-01,505.0,Call,123.69,125.46,0.4772,0.9964,0.0001,-0.0743,0.0111,0.1513
...,...,...,...,...,...,...,...,...,...,...,...,...
125143,2019-02-09,2019-03-29,289.0,Put,19.10,19.51,0.1339,-0.9077,0.0120,-0.0243,0.1580,-0.3551
125144,2019-02-09,2019-03-29,298.0,Call,0.04,0.05,0.1167,0.0115,0.0026,-0.0035,0.0297,0.0041
125145,2019-02-09,2019-03-29,298.0,Put,27.95,28.40,0.1697,-0.9350,0.0071,-0.0228,0.1189,-0.3769
125146,2019-02-09,2019-03-29,310.0,Call,0.01,0.02,0.2195,0.0471,0.0045,-0.0216,0.0973,0.0165


In [ ]:
# underlying
spy_underlying_data = {
    'symbol': 'SPY',
    'period': '2019-01-01 to 2025-07-31',  
    'frequency': 'daily',
    'fields_needed': [
        'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'
    ]
}
spy = yf.download('SPY', start='2019-01-01', end='2025-07-31')

/var/folders/lg/z6wm11r5617_11hqzqgtr_vw0000gn/T/ipykernel_41701/1517287446.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  spy = yf.download('SPY', start='2019-01-01', end='2025-07-31')
[*********************100%***********************]  1 of 1 completed


In [ ]:
def filter_spy_options(spy_df):
    spy_df['date'] = pd.to_datetime(spy_df['date'])
    spy_df['expiration'] = pd.to_datetime(spy_df['expiration'])
    spy_df['days_to_expiry'] = (spy_df['expiration'] - spy_df['date']).dt.days
    spy_df['mid_price'] = (spy_df['bid'] + spy_df['ask']) / 2
    spy_df['bid_ask_spread'] = (spy_df['ask'] - spy_df['bid']) / spy_df['mid_price']
    
    # filters
    filtered = spy_df[
        (spy_df['days_to_expiry'] >= 7) &           
        (spy_df['days_to_expiry'] <= 730) &       
        (spy_df['bid'] > 0) &                       # Valid bid
        (spy_df['ask'] > spy_df['bid']) &           # Valid spread
        (spy_df['bid_ask_spread'] <= 0.05) &        # Spread < 5%
        (spy_df['vol'] > 0.05) &            # IV > 5%
        (spy_df['vol'] < 2.0) &             # IV < 200%
        (spy_df['mid_price'] >= 0.10)               # Min $0.10

    ].copy()
    
    return filtered

# 2. spy underlying
def get_spy_underlying():
    spy_underlying = yf.download('SPY', start='2019-01-01', end='2025-07-31')
    spy_underlying = spy_underlying.reset_index()
    spy_underlying.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    spy_underlying['log_returns'] = np.log(spy_underlying['close'] / spy_underlying['close'].shift(1))
    return spy_underlying

# 3. combine
def prepare_final_dataset(spy_options, spy_underlying):
    # Merge on date
    final_data = spy_options.merge(spy_underlying[['date', 'close']], on='date', how='left')
    final_data.rename(columns={'close': 'spot_price'}, inplace=True)
    
    # log moneyness
    final_data['moneyness'] = np.log(final_data['strike'] / final_data['spot_price'])
    final_data['time_to_expiry'] = final_data['days_to_expiry'] / 252
    
    # add strike/spot filters (80% to 120% moneyness)
    final_data = final_data[
        (final_data['strike'] >= 0.6 * final_data['spot_price']) &
        (final_data['strike'] <= 1.4* final_data['spot_price'])
    ].copy()
    
    return final_data

filtered_spy = filter_spy_options(spy_df)
spy_underlying = get_spy_underlying()
final_dataset = prepare_final_dataset(filtered_spy, spy_underlying)

/var/folders/lg/z6wm11r5617_11hqzqgtr_vw0000gn/T/ipykernel_41701/3285402374.py:37: FutureWarning: YF.download() has changed argument auto_adjust default to True
  spy_underlying = yf.download('SPY', start='2019-01-01', end='2025-07-31')
[*********************100%***********************]  1 of 1 completed


In [ ]:
final_dataset 

,date,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho,days_to_expiry,mid_price,bid_ask_spread,spot_price,moneyness,time_to_expiry
0,2025-07-21,2025-08-01,440.0,Call,188.55,190.28,0.6782,0.9988,0.0000,-0.0533,0.0035,0.1322,11,189.415,0.009133,628.770020,-0.356991,0.043651
1,2025-07-21,2025-08-01,470.0,Call,158.61,160.61,0.5727,0.9984,0.0000,-0.0589,0.0049,0.1412,11,159.610,0.012531,628.770020,-0.291033,0.043651
2,2025-07-21,2025-08-01,505.0,Call,123.69,125.46,0.4772,0.9964,0.0001,-0.0743,0.0111,0.1513,11,124.575,0.014208,628.770020,-0.219207,0.043651
3,2025-07-21,2025-08-01,520.0,Call,108.73,110.58,0.4272,0.9954,0.0002,-0.0791,0.0139,0.1557,11,109.655,0.016871,628.770020,-0.189937,0.043651
4,2025-07-21,2025-08-01,535.0,Call,93.78,95.66,0.3812,0.9936,0.0004,-0.0867,0.0191,0.1598,11,94.720,0.019848,628.770020,-0.161499,0.043651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94781,2019-05-10,2019-06-28,300.0,Call,0.79,0.82,0.1021,0.1471,0.0213,-0.0252,0.2428,0.0559,49,0.805,0.037267,259.478812,0.145107,0.194444
94782,2019-05-10,2019-06-28,300.0,Put,12.75,12.96,0.1159,-0.8193,0.0215,-0.0322,0.2871,-0.2287,49,12.855,0.016336,259.478812,0.145107,0.194444
94783,2019-05-10,2019-06-28,308.0,Put,20.11,20.50,0.1359,-0.9036,0.0120,-0.0239,0.1914,-0.2115,49,20.305,0.019207,259.478812,0.171425,0.194444
94784,2019-05-10,2019-06-28,315.0,Put,27.02,27.42,0.1640,-0.9257,0.0082,-0.0234,0.1538,-0.2032,49,27.220,0.014695,259.478812,0.193898,0.194444


In [ ]:
# rf use 3m treasury
def get_risk_free_rate():
    treasury = yf.download('^IRX', start='2019-01-01', end='2025-07-31')
    treasury = treasury.reset_index()
    treasury = treasury[['Date', 'Close']]
    treasury.columns = ['date', 'risk_free_rate']
    treasury['risk_free_rate'] = treasury['risk_free_rate'] / 100  # Convert percentage to decimal
    return treasury

# SPY dividend yield
def get_spy_dividend_yield():
    spy_info = yf.Ticker('SPY')
    dividend_yield = spy_info.info.get('dividendYield') 
    return dividend_yield/100

def add_market_data(final_dataset):
    risk_free_df = get_risk_free_rate()
    dividend_yield = get_spy_dividend_yield()
    final_dataset = final_dataset.merge(risk_free_df, on='date', how='left')
    final_dataset['dividend_yield'] = dividend_yield
    final_dataset['risk_free_rate'] = final_dataset['risk_free_rate'].fillna(method='ffill')
    
    return final_dataset

complete_dataset = add_market_data(final_dataset)

/var/folders/lg/z6wm11r5617_11hqzqgtr_vw0000gn/T/ipykernel_41701/1775196853.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  treasury = yf.download('^IRX', start='2019-01-01', end='2025-07-31')
[*********************100%***********************]  1 of 1 completed


In [ ]:
complete_dataset 

,date,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho,days_to_expiry,mid_price,bid_ask_spread,spot_price,moneyness,time_to_expiry,risk_free_rate,dividend_yield
0,2025-07-21,2025-08-01,440.0,Call,188.55,190.28,0.6782,0.9988,0.0000,-0.0533,0.0035,0.1322,11,189.415,0.009133,628.770020,-0.356991,0.043651,0.04223,0.0116
1,2025-07-21,2025-08-01,470.0,Call,158.61,160.61,0.5727,0.9984,0.0000,-0.0589,0.0049,0.1412,11,159.610,0.012531,628.770020,-0.291033,0.043651,0.04223,0.0116
2,2025-07-21,2025-08-01,505.0,Call,123.69,125.46,0.4772,0.9964,0.0001,-0.0743,0.0111,0.1513,11,124.575,0.014208,628.770020,-0.219207,0.043651,0.04223,0.0116
3,2025-07-21,2025-08-01,520.0,Call,108.73,110.58,0.4272,0.9954,0.0002,-0.0791,0.0139,0.1557,11,109.655,0.016871,628.770020,-0.189937,0.043651,0.04223,0.0116
4,2025-07-21,2025-08-01,535.0,Call,93.78,95.66,0.3812,0.9936,0.0004,-0.0867,0.0191,0.1598,11,94.720,0.019848,628.770020,-0.161499,0.043651,0.04223,0.0116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88201,2019-05-10,2019-06-28,300.0,Call,0.79,0.82,0.1021,0.1471,0.0213,-0.0252,0.2428,0.0559,49,0.805,0.037267,259.478903,0.145107,0.194444,0.02370,0.0116
88202,2019-05-10,2019-06-28,300.0,Put,12.75,12.96,0.1159,-0.8193,0.0215,-0.0322,0.2871,-0.2287,49,12.855,0.016336,259.478903,0.145107,0.194444,0.02370,0.0116
88203,2019-05-10,2019-06-28,308.0,Put,20.11,20.50,0.1359,-0.9036,0.0120,-0.0239,0.1914,-0.2115,49,20.305,0.019207,259.478903,0.171424,0.194444,0.02370,0.0116
88204,2019-05-10,2019-06-28,315.0,Put,27.02,27.42,0.1640,-0.9257,0.0082,-0.0234,0.1538,-0.2032,49,27.220,0.014695,259.478903,0.193897,0.194444,0.02370,0.0116


In [ ]:
complete_dataset.to_csv('complete_dataset.csv')